[View in Colaboratory](https://colab.research.google.com/github/redcican/Machine-Learning-Basic/blob/master/05_Save_and_Restore_models.ipynb)

Model progress can be saved during - and after - training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work.

#### Options

There are different ways to save TensorFlow models - depending on the API you are using. This guid uses tf.keras.

In [0]:
!pip install -q h5py pyyaml

In [2]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

1.10.0


## 1. Get an example dataset

In [3]:
(train_images, train_labels),(test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


##  2. Define a model

In [4]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    
    return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 3. Save Checkpoints during training



---

The primary use case is to automatically save checkpoints during and at the end of training. This way you can use a trained model without having to retrain it, or pick-up training where you left of - in case the training process was interrupted.

`tf.keras.callbacks.ModelCheckpoint` is a callback that performs this task. The callback takes a couple of argument to configure checkpointing.


### 3.1 Checkpoint callback usage

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs=10,
         validation_data = (test_images,test_labels),
         callbacks=[cp_callback])

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 1.1620 - acc: 0.6670 - val_loss: 0.6813 - val_acc: 0.8070

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 200us/step - loss: 0.4189 - acc: 0.8850 - val_loss: 0.5106 - val_acc: 0.8340

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 197us/step - loss: 0.2845 - acc: 0.9290 - val_loss: 0.4774 - val_acc: 0.8450

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 207us/step - loss: 0.2088 - acc: 0.9470 - val_loss: 0.4714 - val_acc: 0.8480

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 196us/step - loss: 0.1502 - acc: 0.9660 - val_loss: 0.4414 - val_acc: 0.8490

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=======

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Create a new, untrained model. When restoring a model from only weights, you must have a model with the same architecture as the original model. Since it's the same model architecture, we can share weights despite that it's a different instance of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~ 10% accuracy)

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 129us/step
Untrained model, accuracy:  9.00%


Then load the weights from the checkpoint, and re-evaluate

In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 48us/step
Untrained model, accuracy: 86.50%


### 3.2 Checkpoint callback options

The callback provides several options to give the resulting checkpoints unqiue names, and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every 5-epochs.

In [24]:
# include the epoch in the file name. (uses 'str.format')

checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # save weights, every 5-epochs
    period = 5
)

model = create_model()

model.fit(train_images, train_labels,
         epochs=50, callbacks=[cp_callback],
         validation_data = (test_images, test_labels),
         verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [25]:
import pathlib

# Sort the checkpoints by modificaiton time
checkpoints = pathlib.Path(checkpoint_dir).glob("*.index")
checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
checkpoints = [cp.with_suffix('') for cp in checkpoints]
latest = str(checkpoints[-1])
checkpoints

[PosixPath('training_2/cp-0030.ckpt'),
 PosixPath('training_2/cp-0035.ckpt'),
 PosixPath('training_2/cp-0040.ckpt'),
 PosixPath('training_2/cp-0045.ckpt'),
 PosixPath('training_2/cp-0050.ckpt')]

In [26]:
latest

'training_2/cp-0050.ckpt'

In [27]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 113us/step
Restored model, accuracy: 87.50%


### 3.3 What are these files?

The above code stores the weights to a collection of checkpoint-formatted files that contains only the trained weights in a binary format. Checkpoints contain: * One or more shards that contain your model's weights. * An index file that indicates which weights are stored in a which shard.

### 3.4 Manually save weights

In [29]:
# save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 129us/step
Restored model, accuracy: 87.50%


### 3.5 Save the entire model

The entire model can be saved to a file that contains the weight values, the model's configuration, and even the optimizer's configuration. This allow you to checkpoint a model and resume training later - from the exact same state - without access to the original code.

Keras provides a basic save format using the HDF5 standard. For our purpose, the saved model can be treated as a single binar blob.

In [30]:
model = create_model()

model.fit(train_images, train_labels, epochs = 5)

model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 447us/step - loss: 1.2173 - acc: 0.6280
Epoch 2/5
1000/1000 [==============================] - 0s 142us/step - loss: 0.4313 - acc: 0.8820
Epoch 3/5
1000/1000 [==============================] - 0s 158us/step - loss: 0.2886 - acc: 0.9300
Epoch 4/5
1000/1000 [==============================] - 0s 146us/step - loss: 0.1986 - acc: 0.9590
Epoch 5/5
1000/1000 [==============================] - 0s 146us/step - loss: 0.1576 - acc: 0.9660


In [31]:
# Create the model from that file

new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [32]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}".format(100*acc))

1000/1000 [==============================] - 0s 176us/step
Restored model, accuracy: 85.50


This technique saves everything:

* The weight value

* The model's configuration (architecture)

* The optimizer configuration

Keras saves models by inspecting the architeture. Currently, it is not able to save TensorFlow optimizers (from tf.traiin). When using those you will need to re-compile the model after loading, and you will loss the state of the optimizer.